In [8]:
import numpy as np
import pandas as pd
import sys
sys.path.append("/home/shizhenkun/codebase/BioUniprot/tools/")
import funclib
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import linear_model, datasets
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def load_file(path):
    train = pd.read_hdf(path + 'train.h5', key='data')
    train.ec_number = train.ec_number.apply(lambda x: x.replace(',', ''))
    test = pd.read_hdf(path + 'test.h5', key='data')
    test.ec_number = test.ec_number.apply(lambda x: x.replace(',', ''))
    head = funclib.table_head + ['f'+str(i) for i in range(1, 1901)]
    train.columns = head
    test.columns = head
    return train, test

def xgmain(X_train_std, Y_train, X_test_std, Y_test):
    model = XGBClassifier(objective='binary:logistic', random_state=42, use_label_encoder=False, n_jobs=-2, eval_metric='mlogloss')
    model.fit(X_train_std, Y_train.ravel())
    predict = model.predict(X_test_std)
    predictprob = model.predict_proba(X_test_std)
    groundtruth = Y_test
    return groundtruth, predict, predictprob[:,1]

def lrmain(X_train_std, Y_train, X_test_std, Y_test):
    logreg = linear_model.LogisticRegression(solver = 'lbfgs', n_jobs=-2)
    logreg.fit(X_train_std, Y_train)
    predict = logreg.predict(X_test_std)
    lrpredpro = logreg.predict_proba(X_test_std)
    groundtruth = Y_test
    predictprob = lrpredpro[:,1]
    return groundtruth, predict, predictprob

def caculate_metrics(groundtruth, predict, predictprob):
    acc = metrics.accuracy_score(groundtruth, predict)
    precision = metrics.precision_score(groundtruth, predict, zero_division=1 )
    recall = metrics.recall_score(groundtruth, predict)
    f1 = metrics.f1_score(groundtruth, predict)
    auroc = metrics.roc_auc_score(groundtruth, predictprob)
    auprc = metrics.average_precision_score(groundtruth, predictprob)
    tn, fp, fn, tp = metrics.confusion_matrix(groundtruth, predict).ravel()

    npv = tn/(fn+tn+1.4E-45)
    print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', 'auroc','\t\t', 'auprc', '\t\t confusion Matrix')
    print('xgboost', '\t%f' %acc,'\t%f'% precision,'\t\t%f'%npv,'\t%f'% recall,'\t%f'% f1, '\t%f'% auroc,'\t%f'% auprc, '\t', 'tp:',tp,'fp:',fp,'fn:',fn,'tn:',tn)
    
def dtmain(X_train_std, Y_train, X_test_std, Y_test):
    model = tree.DecisionTreeClassifier()
    model.fit(X_train_std, Y_train.ravel())
    predict = model.predict(X_test_std)
    predictprob = model.predict_proba(X_test_std)
    groundtruth = Y_test
    return groundtruth, predict, predictprob[:,1]

def rfmain(X_train_std, Y_train, X_test_std, Y_test):
    model = RandomForestClassifier(oob_score=True, random_state=10, n_jobs=-2)
    model.fit(X_train_std, Y_train.ravel())
    predict = model.predict(X_test_std)
    predictprob = model.predict_proba(X_test_std)
    groundtruth = Y_test
    return groundtruth, predict, predictprob[:,1]

def gbdtmain(X_train_std, Y_train, X_test_std, Y_test):
    model = GradientBoostingClassifier(random_state=10)
    model.fit(X_train_std, Y_train.ravel())
    predict = model.predict(X_test_std)
    predictprob = model.predict_proba(X_test_std)
    groundtruth = Y_test
    return groundtruth, predict, predictprob[:,1]

In [3]:
ori_file_dir = '/home/shizhenkun/codebase/BioUniprot/tasks/task1/data/'
train, test = load_file(ori_file_dir)

In [5]:
res_data=funclib.getblast(train,test)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


In [6]:
isEmzyme_dict = {v: k for k,v in zip(train.isemzyme, train.id )} 
res_data['diamoion_pred'] = res_data['sseqid'].apply(lambda x: isEmzyme_dict.get(x))

blast_res = pd.DataFrame
blast_res = res_data[['id','pident','bitscore', 'diamoion_pred']]

X_train = train.iloc[:,12:]
X_test = test.iloc[:,12:]
Y_train = train.iloc[:,2].astype('int')
Y_test = test.iloc[:,2].astype('int')
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).flatten()
Y_test = np.array(Y_test).flatten()

In [ ]:
groundtruth_lr, predict_lr, predictprob_lr = lrmain (X_train, Y_train, X_test, Y_test)

In [ ]:
groundtruth_xg, predict_xg, predictprob_xg = xgmain(X_train, Y_train, X_test, Y_test)

In [ ]:
groundtruth_dt, predict_dt, predictprob_dt = dtmain(X_train, Y_train, X_test, Y_test)

In [ ]:
groundtruth_rf, predict_rf, predictprob_rf = rfmain(X_train, Y_train, X_test, Y_test)

In [ ]:
groundtruth_gbdt, predict_gbdt, predictprob_gbdt = gbdtmain(X_train, Y_train, X_test, Y_test)

In [ ]:
aa

In [ ]:
methods=['lr', 'xg', 'dt', 'rf', 'gbdt']
print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', 'auroc','\t\t', 'auprc', '\t\t confusion Matrix')
for method in methods:
    funclib.get_integrated_results(res_data, train, test, method)

In [7]:
isEmzyme_dict = {v: k for k,v in zip(train.isemzyme, train.id )} 
res_data['diamoion_pred'] = res_data['sseqid'].apply(lambda x: isEmzyme_dict.get(x))

blast_res = pd.DataFrame
blast_res = res_data[['id','pident','bitscore', 'diamoion_pred']]

X_train = train.iloc[:,12:]
X_test = test.iloc[:,12:]
Y_train = train.iloc[:,2].astype('int')
Y_test = test.iloc[:,2].astype('int')
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).flatten()
Y_test = np.array(Y_test).flatten()

In [ ]:
funclib.run_integrated(res_data, train, test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix


0it [00:00, ?it/s]/home/shizhenkun/codebase/BioUniprot/tools/funclib.py:236: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else:
/home/shizhenkun/codebase/BioUniprot/tools/funclib.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
46129it [00:24, 1920.05it/s]


baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
xg 	 0.935615  	0.919272 		 0.949210  	0.937716	 0.928403		 	 	 	 tp:19256  fp:1691  fn:1279  tn:23903


In [ ]:
aa